In [1]:
# write_to_db.py

import pandas as pd
import sqlite3
from sqlalchemy import create_engine, text
from sqlalchemy.orm import Session

#  1. Читаем учетные данные из SQLite 
sqlite_path = r"C:\Users\malin\Downloads\creds.db"
conn = sqlite3.connect(sqlite_path)
cursor = conn.cursor()
cursor.execute("SELECT url, port, user, pass FROM access LIMIT 1")
db_url, db_port, db_user, db_password = cursor.fetchone()
conn.close()

# 2. Подключаемся к PostgreSQL 
db_name = "homeworks"
engine = create_engine(
    f"postgresql+psycopg2://{db_user}:{db_password}@{db_url}:{db_port}/{db_name}"
)


In [2]:

# 3. Загружаем свой датасет 
df = pd.read_parquet(r"C:\Users\malin\my_project\my_project.parquet")
df = df.head(100)  # максимум 100 строк
df.columns = [col.lower() for col in df.columns]

# 4. Записываем данные в таблицу с вашей фамилией 
my_table_name = "gabdrakhmanova"  
df.to_sql(
    name=my_table_name,
    con=engine,
    schema="public",
    if_exists="replace",
    index=False
)

#  5. Добавляем комментарий на таблицу 
with engine.begin() as conn:
    conn.execute(
    text(f"COMMENT ON TABLE public.{my_table_name} IS 'Таблица с анкетами и выборкой смартфон-зависимости';")
)


print(f"Данные успешно загружены в таблицу {my_table_name}")

Данные успешно загружены в таблицу gabdrakhmanova


In [3]:
#  6. Проверяем первые 5 строк таблицы 
with Session(engine) as session:
    result = session.execute(text(f"SELECT * FROM public.{my_table_name} LIMIT 5"))
    rows = result.fetchall()
    print("\nПервые 5 строк таблицы:")
    for row in rows:
        print(row)



Первые 5 строк таблицы:
(2208, None, 23, 2, 1, None, 1, 2, 'тат', True, 0, 3, None, 1, '1, 1 г', False, '[0]', '0', 'nan', 0, 'nan', True, False, '1', 2, 1, 'nan', 'nan', 'nan', 'nan', '4,5,9', True, 'AA2790', 21, 'V1058', 1, 0, 1, 1, 1, 1, 1, 1, None, 2, 2, 1, 2, 0, 1, 1)
(2197, '14.09.2000', 21, 2, 1, None, 3, 19, 'остальные', False, -9, 2, None, 1, '0', False, '[0]', '0', 'nan', 1, 'nan', False, True, '0', 2, 1, '[2]', 'кашель', 'nan', 'nan', '[10]', True, 'AA3194', 50, 'V1059', 1, 0, 0, 2, 2, 0, 2, 2, 2, 2, 1, 0, 1, 1, 1, 1)
(2259, '08.09.2000', 21, 2, 1, None, 3, 19, 'тат', True, 2, 2, None, 1, '1', False, '[0]', '2', 'nan', 0, 'nan', True, None, 'nan', 2, 1, '[2,5]', 'боли в голове, сухой кашель', '1', 'nan', '[10]', True, 'AB9398', 26, 'V1060', 0, 0, 1, 2, 2, 0, 0, 2, 1, 2, 2, 1, 2, 2, 2, 1)
(2568, '10/10/2000', 21, 2, 3, None, 3, None, 'тат', False, None, 2, None, 1, '1, до 1г', False, '[0]', '0', 'nan', 0, 'nan', True, False, '0', 2, 1, '[2]', 'сильные боли в голове', 'nan', 